---
title: 多水平结构方程模型 Multilevel SEM 理论和应用教程
date: 2024-03-18 11:27:59
tags: [mplus, sem]
mathjax: true
---

本篇教程先介绍多水平结构方程的基本概念, 有一些数学符号可能难以理解, 
但是不要担心, 本教程不是给专家看的, 所以我们针对入门学生的一些常见问题进行讲解,
即便无法理解理论部分, 只要你遵循我们的指导也可以完成一篇合格的多水平结构方程的分析报告.

本篇教程主要使用mplus软件, Mplus 可能是好用的多水平结构方程软件，尽管 EQS 和 LISREL 中也有类似的功能。 此外，R 中的 OpenMx 包是免费的，支持多级分析，但需要采用截然不同的语法和规范方法, 适合会R语言的童鞋.

<!-- more -->

这篇教程虽然不是针对会R的童鞋的, 但是我们建议你使用 R 的一个工具库 MplusAutomation, 这个工具库帮助你完成非常无聊的结果整理工作, 因为 Mplus 输出的结果是文本文件, 难以转换为表格, 难以对比不同模型的结果, 所以我们鼓励你使用 MplusAutomation 这个工具, 它可以自动帮你完成这个功能. 如果对R感兴趣, 可以看我们录制的R语言的快速入门视频教程, 因为学习编程的诀窍只有一个, 就是快速入门, 快速应用, 视频放在这里[《R语言快速入门教程》](https://www.bilibili.com/video/BV1Wt4y1g7rn/)

## 什么是多水平模型(不限于多水平结构方程模型)

从一般的线性模型开始, 然后逐渐引入多水平的概念. 你可以看下一般的线性回归模型:

$$
\begin{align*}
Y &= B_0 + B_1 X_1 + B_2 X_2 + ... B_p X_p + \varepsilon \\
\varepsilon &\sim \mathcal{N}(0, \sigma^2)
\end{align*}
$$

一般的线性模型理解起来非常简单, p个预测变量x, 通过线性组合, 来对被预测的变量Y进行预测. 但是这个模型有一个前提, 就是样本之间没有相互依赖, 
或者说, 回归残差ε是独立同分布的. 但是如果你的样本是重复测量的, 你可能会得到下面这样的样本数据:

重复测量的数据格式通常包括两种主要类型：长型格式和宽型格式。

- 长型格式：在这种格式中，每个受试者的每次重复测量都会作为一条独立的记录。例如，如果一个受试者被重复测量了四次，那么该受试者就会有四条记录。每条记录都包含受试者的标识信息以及对应的测量值。这种格式的优点在于可以清晰地展示每次测量的具体情况，但可能会导致数据集的行数较多。
- 宽型格式：这是数据分析时最常用的格式。在这种格式中，每个受试者只有一条记录，而重复测量的数据则作为该记录的不同变量。例如，如果一个受试者被重复测量了四次，那么这四个测量值就会作为该受试者的四个不同变量。这种格式的优点在于可以简化数据集的结构，方便进行统计分析，但可能需要对数据进行适当的转换和重构。

如果是长型格式 , 你看下面的数据, 同一个被试会在不同的时间点测量4次, 那么同一个被试的4个样本是高度相关的, 这时候不可以直接使用一般线性回归分析.

| 被试ID | 测量轮次 | 测量值 |  
| :--: | :--: | :--: |  
| 001 | 1 | 25.5 |  
| 001 | 2 | 26.2 |  
| 001 | 3 | 25.8 |  
| 001 | 4 | 26.1 |  
| 002 | 1 | 24.9 |  
| 002 | 2 | 25.3 |  
| 002 | 3 | 25.1 |  
| 002 | 4 | 25.5 |  
| ... | ... | ... |

常见的多水平数据还有下面几种:

- 嵌套在个体中的实验任务的试验（例如，反应时间或准确性数据） 
- 在每只动物中测量一组神经元（例如，群体），但目的是描述一组动物的模式 
- 问卷或调查的纵向测量 
- 基于访谈的数据 
- 同一个班级的学生
- 治疗组内的患者

在每种情况下，多水平分析（包括多水平 SEM）的目标是明确解释簇内和簇间的变异性。 如果我们要分析数据而忽略群体结构，我们就会高估自由度（因为我们的观察是非独立的,没有那么自由）。 我们无法区分簇内和簇间的影响； 相反，我们的参数将反映两种效应的混合, 这使得我们得出错误的结论.

## 水平间效应

多水平分析还允许研究人员在模型的每个水平提出具体问题。 让我们考虑一下学生 (L1层面) 嵌套在学校 (L2层面) 内的情况。 在学校层面，我们假设学生与教师的比例会影响阅读成绩，因此每位教师的学生越多，成绩越差。 根据定义，学生与教师的比例仅作为学校层面的变量才有意义（因为同一所学校的学生有类似的师生比例, 这个变量不随学校内学生的变化而变化），在个人层面上没有变化。 这可能看起来像下面这样的数据:

| student | school | stratio | readtest |  
| :---: | :---: | :---: | :---: |  
| 1 | Radio Park | 20 | 15 |  
| 2 | Radio Park | 20 | 12 |  
| 3 | Radio Park | 20 | 19 |  
| 4 | Corl Street | 16 | 24 |  
| 5 | Corl Street | 16 | 27 |  
| 6 | Corl Street | 16 | 21 |


- 第一列 student 是学生的编号。
- 第二列 school 是学生所在的学校名称。
- 第三列 stratio 是学生所在学校的师生比，即每个老师负责的学生数量。从数据中可以看到，Radio Park 学校的师生比是20，而 Corl Street 学校的师生比是16。
- 第四列 readtest 是学生的阅读测试分数。


请注意，stratio在同一个学校下没有变化。 这是预测变量最简单的情况，因为不用担心它可能代表学生和学校效应的混合。 也就是说，stratio 无法预测任何 L1 方差(即同一个学校内学生阅读成绩的变化不可能是师生比决定的)。

用方程来表示变量的关系就是:

$$
\begin{align*}
\textrm{readtest}_{ij} &= \beta_{0j} + r_{ij} \\
\beta_{0j} &= \gamma_{00} + \gamma_{01} \textrm{stratio}_j + \mu_{0j} \\
\end{align*}
$$

第一个方程没有预测变量(自变量), j是学校代码, i是学生代码, 那么 $\beta_{0j}$ 意思是学校的阅读成绩平均分, 同一个学校的学生有同一个值, 
$r_{ij}$是学生的离均差, 即偏离所在学校的平均值 $\beta_{0j}$ 的大小. 第二个方程 $\gamma_{00}$ 是所有学校所有同学的总平均分, 
师生比会影响到学校的平均成绩， 师生比的效应是 $gamma_{01}$ .  这个模型没有纳入任何个体层面的变量， 但是这个模型是帮助你理解什么是多水平模型的：
师生比通过影响学校平均水平$beta_{0j}$ 进而影响到了学生的成绩{readtest}_{ij}。 

下面

https://psu-psychology.github.io/psy-597-SEM/15_multilevel/multilevel_sem.html#what-is-multilevel-analysis